In [23]:
import random
import torch
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import torch.nn.functional as F

torch.manual_seed(42)

## Various sine function

In [25]:
T = 0.1024
fs = 5000
# meaning n_samples = T * fs = 0.1024 * 5000 = 512 (same as our windowed data)
frequencies = [20, 90]  # Hz
t = torch.arange(0, T, 1 / fs)
L = len(t)

#### sum of sines

In [24]:
sine_wave = torch.zeros_like(t)
for freq in frequencies:
	sine_wave += torch.sin(2 * np.pi * freq * t)

#### changing freq

In [26]:
transition_point = int(0.5 * L)  # Number of samples before frequency transition
changing_sine = torch.zeros_like(t)
changing_sine[:transition_point] = torch.sin(2 * torch.pi * frequencies[0] * t[:transition_point])
changing_sine[transition_point:] = torch.sin(2 * torch.pi * frequencies[-1] * t[transition_point:])

#### chirp

In [27]:
f0 = frequencies[0]  # starting frequency in Hz
fn = frequencies[-1]  # ending frequency in Hz
t = torch.linspace(0, T, int(T * fs), dtype=torch.float32)
omega = 2 * torch.pi * (f0 + (fn - f0) * t / T)
chirp = torch.sin(omega * t)

#### real force data

In [28]:
forces = torch.load(r'G:\My Drive\Master\Lab\Thesis\Results\22_11_2023\f19+f23_list_clean.pt')

In [29]:
random_force_index = random.randint(0, len(forces) - 1)
f_dataset = forces[random_force_index]
window_size = int(T * fs)
window_start = random.randint(0, len(f_dataset) - window_size)
window_end = window_start + window_size
window = f_dataset[window_start:window_end, :]
f1, f2, f3, f4 = window[:, 0], window[:, 1], window[:, 2], window[:, 3]
print('dataset index:', random_force_index, f'(out of {len(forces) - 1})')
print(f'window index: [{window_start}, {window_start + 1},...,{window_end}] (size {window_size})',
	  f'within the dataset with indices [0,1,...,{len(f_dataset) - 1}] (size {len(f_dataset)})')


dataset index: 44 (out of 152)
window index: [1913, 1914,...,2425] (size 512) within the dataset with indices [0,1,...,3268] (size 3269)


In [30]:
class Dataset:
	def __init__(self, data, name, fs):
		self.name = name
		self.raw_data = data
		self.fft = None
		self.raw_fft = None
		self.fftfreq = None
		self.raw_fftfreq=None
		self.stft = None
		self.fs = fs
		self.data = self._parse_data(data)

	def __str__(self):
		if self.name == 'sine':
			return r'$f(t)=$' + '+'.join([rf'$sin(2\pi\cdot{f}t)$' for f in frequencies])
		else:
			return self.name

	def _parse_data(self, data):
		data = torch.from_numpy(signal.savgol_filter(
			x=data,
			window_length=5,
			polyorder=3
		))
		"""
		the number of fft bins is nyquist = fs/2 = 2500. this means that len(data) points are spread across 2500 bins.
		we want to have at least 2500 points, so we pad the data with zeros to reach that number.
		but through experiments i can tell that this is still nout enough, so i double that number in the power of two.
		"""
		# data = data * torch.hann_window(len(data))
		padding_size = self.fs // 2 - len(data)
		# round padding_size to the nearest power of two for fast FFT:
		padding_size = 2 ** (int(np.ceil(np.log2(padding_size))) + 1)
		data = F.pad(data, (0, padding_size), mode='constant', value=0)
		# data = data * torch.hann_window(len(data))
		return data

	def calc_fft(self, apply_abs=True, trim_negative=True, shapes=True):
		for i, data in enumerate([self.data, self.raw_data]):
			fft = torch.fft.fft(data)
			freqs = torch.fft.fftfreq(len(data), 1 / self.fs)
			if trim_negative:
				freqs_mask = freqs >= 0
				freqs = freqs[freqs_mask]
				fft = fft[freqs_mask]
			if apply_abs:
				fft = torch.abs(fft)
			if i == 0:
				self.fft = fft
				self.fftfreq = freqs
			elif i == 1:
				self.raw_fft = fft
				self.raw_fftfreq=freqs
			if shapes:
				print(fft.shape)

	def calc_stft(self, n_fft=200, hop_length=None, win_length=None, pad_mode='reflect', apply_abs=True, shapes=True):
		if win_length is None:
			win_length = n_fft
		if hop_length is None:
			hop_length = n_fft // 4
		stft = torch.stft(
			input=self.data,
			n_fft=n_fft,
			hop_length=hop_length,  #default n_fft/4
			win_length=win_length,  #default n_fft
			window=torch.hann_window(win_length),  # hann is a cosine window with ends that touch zero
			center=False,
			pad_mode=pad_mode,
			normalized=False,  # not normalized
			onesided=True,  # only return the positive frequencies
			return_complex=True  # return complex numbers
		)
		if apply_abs:
			stft = torch.abs(stft)
		self.stft = stft
		if shapes:
			print("Spectrogram shape is (B?, N, T, C?), where:")
			print(f' - B --> None')
			print(f' - N --> (n_fft//2)+1=({n_fft}//2)+1={(n_fft // 2) + 1}')
			print(
				f" - T --> 1+(L-n_fft)//hop_length=1+({len(self.data)}-{n_fft})//{hop_length}={1 + (len(self.data) - n_fft) // hop_length}")
			print(f" - C --> None")
			print(f'STFT shape: {stft.shape}')



In [31]:
dataset = Dataset(f4, 'f4', fs=fs)
dataset.calc_fft()
n_fft = int(0.25 * len(dataset.data))
hop_length = n_fft // 32
window_len = n_fft // 16
dataset.calc_stft(n_fft=n_fft, hop_length=hop_length)

torch.Size([2304])
torch.Size([256])
Spectrogram shape is (B?, N, T, C?), where:
 - B --> None
 - N --> (n_fft//2)+1=(1152//2)+1=577
 - T --> 1+(L-n_fft)//hop_length=1+(4608-1152)//36=97
 - C --> None
STFT shape: torch.Size([577, 97])


In [32]:
k = 2
sorted_fft = torch.sort(dataset.fft, descending=True)
top_k_freqs = dataset.fftfreq[sorted_fft.indices[:k]]

In [ ]:
import matplotlib

matplotlib.use('TkAgg')
plt.figure(figsize=(12, 10))

# Subplot 1: Input signal
plt.subplot(3, 1, 1)
plt.scatter(torch.arange(len(dataset.data)), dataset.data, s=5, label='filtered data')
plt.scatter(torch.arange(len(dataset.raw_data)), dataset.raw_data, s=5, label='raw data')
plt.title(r'$f(t)$' + ' ,shape: ' + str(dataset.data.shape)), plt.xlabel('# Sample'), plt.ylabel('Amplitude')
plt.grid(), plt.legend()

# Subplot 2: FFT
plt.subplot(3, 1, 2)
plt.scatter(dataset.fftfreq, dataset.fft, s=10, label='FFT[filtered data]')
plt.scatter(dataset.raw_fftfreq, dataset.raw_fft, s=10, label='FFT[raw data]')
plt.title(r'$|FFT[f(t)]|^+$' + ' ,shape: ' + str(dataset.fft.shape)), plt.xlabel('Frequency (Hz)'), plt.ylabel(
	'Magnitude'), plt.grid()
# ax1 = plt.gca()
# ax2= ax1.secondary_xaxis('top')

for i, f in enumerate(top_k_freqs):
	plt.vlines(x=f, ymin=0, ymax=max(dataset.fft), color='black', linestyle='-.',
			   label=rf'$f_{i}={f:.3f}$ Hz')
# plt.xlim(0, 200)
plt.legend()

# Subplot 3: Spectrogram
plt.subplot(3, 1, 3)
stft_np = dataset.stft.numpy()
plt.imshow(stft_np, aspect='auto', cmap='viridis', origin='lower', extent=[0, stft_np.shape[1], 0, fs / 2])
plt.colorbar(label='Magnitude')
plt.title('Spectrogram' + ' ,shape: ' + str(dataset.stft.shape)), plt.xlabel('Time (s)'), plt.ylabel('Frequency (Hz)')
# plt.ylim(0, max(top_k_freqs) + 100)
# plt.ylim(0,30)
plt.tight_layout()
plt.show()

